In [1]:
%autosave 60
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

Autosaving every 60 seconds


## Auxiliary Functions

We need the parser for first order formulas, hence we import it.

In [2]:
import folParser as fp

def parse(s):
    return fp.LogicParser(s).parse()

The resolution calculus works with clauses.  The module <tt>folCNF</tt> implements the function $\texttt{normalize}(f)$ that turns a formula $f$ into a set of clauses.

In [3]:
import folCNF as cnf

The module <tt>unify</tt> implements unification.

In [4]:
import unify

In [30]:
def arb(S):
    for x in S:
        return x

Given a literal $l$ the function $\texttt{complement}(l)$ computes the complement $\overline{\,l\,}$ of the literal $l$.

In [5]:
def complement(l):
    "Compute the complement of the literal l."
    if l[0] == '¬':
        return l[1]
    else:
        return ('¬', l)

In [6]:
complement(('p', 'x'))

('¬', ('p', 'x'))

In [7]:
complement(('¬', ('p', 'x')))

('p', 'x')

Given a clause $C$, the function $\texttt{collectVariables}(C)$ computes the set of all variables occurring in $C$.  The function $\texttt{collectVariables}$ can also collect the variables in a literal or a term.

In [8]:
def collectVariables(C):
    if isinstance(C, frozenset):
        return { x for literal in C 
                   for x in collectVariables(literal) 
               }
    if isinstance(C, str): # C is a variable
        return { C }
    if C[0] == '¬':
        return collectVariables(C[1])
    # C must be a term or atomic formula
    args = C[1:]
    return { x for t in args for x in collectVariables(t) }

In [9]:
s = '∀g:∀c:(Grandparent(g, c) ↔ ∃p: (Parent(g, p) ∧ Parent(p, c)))'
f = parse(s)
f

('∀',
 'g',
 ('∀',
  'c',
  ('↔',
   ('Grandparent', 'g', 'c'),
   ('∃', 'p', ('∧', ('Parent', 'g', 'p'), ('Parent', 'p', 'c'))))))

In [10]:
Clauses = cnf.normalize(f)
Clauses

{frozenset({('Parent', ('sk1', 'g', 'c'), 'c'),
            ('¬', ('Grandparent', 'g', 'c'))}),
 frozenset({('Parent', 'g', ('sk1', 'g', 'c')),
            ('¬', ('Grandparent', 'g', 'c'))}),
 frozenset({('Grandparent', 'g', 'c'),
            ('¬', ('Parent', 'g', 'z')),
            ('¬', ('Parent', 'z', 'c'))})}

In [11]:
for C in Clauses:
    print(f'collectVariables({C}) = \n\t{collectVariables(C)}')

collectVariables(frozenset({('Parent', ('sk1', 'g', 'c'), 'c'), ('¬', ('Grandparent', 'g', 'c'))})) = 
	{'g', 'c'}
collectVariables(frozenset({('¬', ('Grandparent', 'g', 'c')), ('Parent', 'g', ('sk1', 'g', 'c'))})) = 
	{'g', 'c'}
collectVariables(frozenset({('¬', ('Parent', 'g', 'z')), ('Grandparent', 'g', 'c'), ('¬', ('Parent', 'z', 'c'))})) = 
	{'g', 'c', 'z'}


In [12]:
import string

def renameVariables(f):
    Variables = collectVariables(f)
    NewVars   = set(string.ascii_lowercase) - Variables
    NewVars   = list(NewVars)
    sigma     = { x: NewVars[i] for (i, x) in enumerate(Variables) }
    return unify.apply(f, sigma)

In [13]:
for C in Clauses:
    print(f'renameVariables({C}) = \n\t{renameVariables(C)}')

renameVariables(frozenset({('Parent', ('sk1', 'g', 'c'), 'c'), ('¬', ('Grandparent', 'g', 'c'))})) = 
	frozenset({('¬', ('Grandparent', 'z', 'b')), ('Parent', ('sk1', 'z', 'b'), 'b')})
renameVariables(frozenset({('¬', ('Grandparent', 'g', 'c')), ('Parent', 'g', ('sk1', 'g', 'c'))})) = 
	frozenset({('Parent', 'z', ('sk1', 'z', 'b')), ('¬', ('Grandparent', 'z', 'b'))})
renameVariables(frozenset({('¬', ('Parent', 'g', 'z')), ('Grandparent', 'g', 'c'), ('¬', ('Parent', 'z', 'c'))})) = 
	frozenset({('Grandparent', 'b', 'y'), ('¬', ('Parent', 'q', 'y')), ('¬', ('Parent', 'b', 'q'))})


## Some Formulas for Testing

In [14]:
s1 = '∀x:(∀y:(Child(y, x) → CanFly(y)) → Happy(x))'
s2 = '∀x:(Red(x) → CanFly(x))'
s3 = '∀x:(Red(x) → ∀y:(Child(y, x) → Red(y)))'
s4 = '¬∀x:(Red(x) → Happy(x))'
f1 = parse(s1)
f2 = parse(s2)
f3 = parse(s3)
f4 = parse(s4)
Clauses = cnf.normalize(f1) | cnf.normalize(f2) | cnf.normalize(f3) | cnf.normalize(f4)
C1, C2, C3, C4, C5, C6 = tuple(Clauses)

In [15]:
C1

frozenset({('Red', ('sk3',))})

In [16]:
C2

frozenset({('Red', 'y'), ('¬', ('Child', 'y', 'x')), ('¬', ('Red', 'x'))})

In [17]:
C3

frozenset({('Child', ('sk2', 'x'), 'x'), ('Happy', 'x')})

In [18]:
C4

frozenset({('¬', ('Happy', ('sk3',)))})

In [19]:
C5

frozenset({('Happy', 'x'), ('¬', ('CanFly', ('sk2', 'x')))})

In [20]:
C6

frozenset({('CanFly', 'x'), ('¬', ('Red', 'x'))})

# A Calculus for First Order Logic

Given a two clauses <tt>C1</tt> and <tt>C2</tt>, the function $\texttt{resolve}(\texttt{C1}, \texttt{C2})$ computes a set of all clauses that can be inferred from <tt>C1</tt> and <tt>C2</tt> by applying the resolution rule.

In [25]:
def resolve(C1, C2):
    C2 = renameVariables(C2)
    Result = set()
    for L1 in C1:
        for L2 in C2:
            mu = unify.unify(L1, complement(L2))
            if mu != None:
                C1C2 = unify.apply((C1 - { L1 }) | (C2 - { L2 }), mu)
                Result.add(C1C2)
    return Result

In [26]:
resolve(C1, C2)

{frozenset({('Red', 'b'), ('¬', ('Child', 'b', ('sk3',)))})}

In [42]:
def factorize(C):
    Result = set()
    for L1 in C:
        for L2 in C:
            if L1 != L2:
                mu  = unify.unify(L1, L2)
                if mu != None:
                    Cmu = unify.apply(C, mu)
                    Result.add(Cmu)
    return Result

In [43]:
C7 = arb(cnf.normalize(parse('∀x:∀y:P(F(x),y) ∨ ∀u:∀v:P(u,G(v))')))
C7

frozenset({('P', ('F', 'x'), 'y'), ('P', 'u', ('G', 'v'))})

In [44]:
factorize(C7)

{frozenset({('P', ('F', 'x'), ('G', 'v'))})}

In [48]:
C8 = arb(cnf.normalize(parse('∀x:∀y:(¬P(F(x),y)) ∨ ∀u:∀v:(¬P(u,G(v)))')))
C8

frozenset({('¬', ('P', 'u', ('G', 'v'))), ('¬', ('P', ('F', 'x'), 'y'))})

In [49]:
factorize(C8)

{frozenset({('¬', ('P', ('F', 'x'), ('G', 'v')))})}